In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
num_rows = 0

train_df = np.empty((0,59), dtype=np.float64)

for i in range(1, 11):
  l = [2,5,9]
  if i not in l:
    directory = 'merged files/' + str("{:03d}".format(i) + '.csv')
    df_chunk = pd.read_csv(directory).iloc[:, 1:]
    train_df = np.vstack((train_df, df_chunk.to_numpy(dtype=np.float64)))
    num_rows += len(df_chunk)
    print(df_chunk.shape, train_df.shape)
print(train_df.shape)

(376504, 59) (376504, 59)
(1034504, 59) (1411008, 59)
(322005, 59) (1733013, 59)
(530504, 59) (2263517, 59)
(343004, 59) (2606521, 59)
(884009, 59) (3490530, 59)
(803504, 59) (4294034, 59)
(4294034, 59)


In [3]:
X = train_df[:, :-1]
Y = train_df[:, -1]
X.shape, Y.shape

((4294034, 58), (4294034,))

In [4]:
num_columns = 59


test_result_array = np.empty((0, num_columns), dtype=np.float64)

for i in range(11, 13):
    directory = 'merged files/' + str("{:03d}".format(i)) + '.csv'
    df_chunk = pd.read_csv(directory).iloc[:, 1:]
    test_result_array = np.vstack((test_result_array, df_chunk.to_numpy(dtype=np.float64)))


print(test_result_array.shape)

(1099508, 59)


In [5]:
X_ = test_result_array[:, :-1]
Y_ = test_result_array[:, -1]
X_.shape, Y_.shape

((1099508, 58), (1099508,))

In [6]:
#

In [7]:
import xgboost as xgb

dtrain_reg = xgb.DMatrix(X, Y)
dtest_reg = xgb.DMatrix(X_, Y_)

In [8]:
params = {
    'objective': 'binary:logistic',  # Binary classification
    'eval_metric': 'logloss',        # Evaluation metric
    'max_depth': 25,                # Maximum depth of a tree
    'eta': 0.15,                      # Learning rate
    'subsample': 0.8,                # Subsample ratio of the training instances
    'colsample_bytree': 0.8,         # Subsample ratio of columns when constructing each tree
    'seed': 42,                      # Random seed

}

In [9]:
xgb_classifier = xgb.train(params, dtrain_reg)

In [10]:
# Make predictions on the test set
y_pred = xgb_classifier.predict(dtest_reg)

In [11]:
y_pred_bool = [0 if x < 0.78 else 1 for x in y_pred]

In [12]:
# Evaluate the model's performance
accuracy = accuracy_score(Y_, y_pred_bool)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6606


In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_, y_pred_bool)
cm

array([[709754,  30254],
       [342943,  16557]], dtype=int64)